## 🏠 House Prices -contexto

Este notebook apresenta a avaliação de modelos lineares aplicados ao desafio *House Prices – Advanced Regression Techniques do Kaggle*. O foco aqui é estabelecer uma base sólida de experimentos, utilizando um pré-processador padronizado (`carregado via preprocessador_v0.joblib`) e pipelines consistentes para garantir reprodutibilidade e ausência de data leakage.

A estrutura foi organizada para manter clareza entre carregamento dos dados, preparação das variáveis, construção dos modelos e avaliação dos resultados.

###  Principais Highlights

* **Pré-processamento padronizado:** carregado via `preprocessador_v0.joblib`, garantindo consistência entre notebooks e evitando data leakage.

* **Pipeline único para cada modelo:** integrando automaticamente tratamento de dados e etapa de regressão.

* **Avaliação completa** com:
    * Métricas no conjunto de teste (MAE, RMSE, R²)

    * **Validação cruzada** (K-Fold) com análise da variação entre folds.

* Testes realizados para 5 modelos lineares:

    * Linear Regression

    * Ridge (default e versão ajustada)

    * Lasso (default e versão ajustada)

* Funções auxiliares (`metricas_model()` e `valida()`) padronizam a avaliação e simplificam a comparação entre experimentos.


## 1. Bibliotecas


In [1]:
# =====================================================
# 🏠 House Prices - Modelos Lineares
# =====================================================
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, 
                       message='Found unknown categories in columns')
import time
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

# Scikit-learn - Model selection e avaliação
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV

# Scikit-learn - Pré-processamento e pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Scikit-learn - Modelos lineares
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV

# Scikit-learn - Métricas de avaliação
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Distribuições para busca de hiperparâmetros
from scipy.stats import randint, uniform,loguniform

from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from joblib import dump,load

## 2. Dataload & preprocessamento

In [2]:
# =====================================================
# 📁 1. Leitura dos dados
# =====================================================
dfo = pd.read_csv("/home/akel/PycharmProjects/Kaggle/HousePrices/data/train.csv")
df_train=dfo.copy()
# =====================================================
# ⚙️ 2. carregamento dos preprocessador 
# =====================================================
preprocessador0 = load("preprocessador_v0.joblib")

# =====================================================
# 🧹 3. Pré-processamento inicial
# =====================================================
# remoção de colunas com muitos nulos (> 10%)
colnull_train=df_train.columns[(df_train.isnull().sum()/df_train.shape[0]>0.1)] # 
df_train=df_train.drop(columns=colnull_train,axis=1)
id_train=df_train['Id']

# =====================================================
#  🤖 4.Modelos
# =====================================================
model_lr = LinearRegression()
LR       = pipe_models(model_lr, preprocessador0)

model_rd0 = Ridge(alpha=0.01)
RD0       = pipe_models(model_rd0, preprocessador0)

model_rd1 = Ridge(alpha=19.30, max_iter=1000, random_state=42, solver='sag',
                 tol=0.001)
RD1       = pipe_models(model_rd1, preprocessador0)

model_ls0 = Lasso(tol=0.001)
LS0       = pipe_models(model_ls0, preprocessador0)

model_ls1 = Lasso(alpha=0.002043, max_iter=2000, tol=0.001)
LS1       = pipe_models(model_ls1, preprocessador0)


## 3.Treinamento

In [3]:
X=df_train.drop(['Id', 'SalePrice'], axis=1)
y_log=np.log1p(df_train['SalePrice'])
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.3, random_state=42)
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)

res=metricas_model(y_test, y_pred, 'LR')
r2=valida(X,y_log,model=LR,N=7)

🤖 LR
MAE:    0.095
RMSE: 0.1826
R²:   0.8035
validação cruzada (K-Fold Cross Validation)
Fold 1: R² = 0.9228
Fold 2: R² = 0.6297
Fold 3: R² = 0.8888
Fold 4: R² = 0.5765
Fold 5: R² = 0.8656
Fold 6: R² = 0.9123
Fold 7: R² = 0.6567

📊 R² médio: 0.7789 ± 0.1395


In [4]:
RD0.fit(X_train, y_train)
y_pred = RD0.predict(X_test)

res=metricas_model(y_test, y_pred, 'RD-0')
r2=valida(X,y_log,model=RD0,N=7)

🤖 RD-0
MAE:   0.0945
RMSE: 0.1679
R²:   0.8337
validação cruzada (K-Fold Cross Validation)
Fold 1: R² = 0.9243
Fold 2: R² = 0.6977
Fold 3: R² = 0.8879
Fold 4: R² = 0.5775
Fold 5: R² = 0.8709
Fold 6: R² = 0.9118
Fold 7: R² = 0.7182

📊 R² médio: 0.7983 ± 0.1238


In [6]:
RD1.fit(X_train, y_train)
y_pred = RD1.predict(X_test)

res=metricas_model(y_test, y_pred, 'RD-1')
r2=valida(X,y_log,model=RD1,N=7)

🤖 RD-1
MAE:   0.0963
RMSE: 0.1346
R²:   0.8932
validação cruzada (K-Fold Cross Validation)
Fold 1: R² = 0.9048
Fold 2: R² = 0.9021
Fold 3: R² = 0.8948
Fold 4: R² = 0.5777
Fold 5: R² = 0.9114
Fold 6: R² = 0.9089
Fold 7: R² = 0.8894

📊 R² médio: 0.8556 ± 0.1137


In [8]:
LS0.fit(X_train, y_train)
y_pred = LS0.predict(X_test)

res=metricas_model(y_test, y_pred, 'LS-0')
r2=valida(X,y_log,model=LS0,N=7)

🤖 LS-0
MAE:   0.3188
RMSE: 0.4122
R²:   -0.0015
validação cruzada (K-Fold Cross Validation)
Fold 1: R² = -0.0032
Fold 2: R² = -0.0010
Fold 3: R² = -0.0057
Fold 4: R² = -0.0077
Fold 5: R² = -0.0114
Fold 6: R² = -0.0019
Fold 7: R² = -0.0089

📊 R² médio: -0.0057 ± 0.0036


In [11]:
LS1.fit(X_train, y_train)
y_pred = LS1.predict(X_test)

res=metricas_model(y_test, y_pred, 'LS-1')
r2=valida(X,y_log,model=LS1,N=7)

🤖 LS-1
MAE:   0.0994
RMSE: 0.1384
R²:   0.8871
validação cruzada (K-Fold Cross Validation)
Fold 1: R² = 0.8917
Fold 2: R² = 0.8998
Fold 3: R² = 0.8910
Fold 4: R² = 0.5433
Fold 5: R² = 0.8963
Fold 6: R² = 0.9143
Fold 7: R² = 0.8824

📊 R² médio: 0.8455 ± 0.1237
